In [1]:
## 크롤링 및 DB연동을 위한 패키지 import

import cx_Oracle
import pandas as pd
import numpy as np
import requests as rq
import time
from tqdm import tqdm_notebook as tn
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys

In [2]:
# 데이터베이스 연동

dsn = cx_Oracle.makedsn('121.147.213.53',1521,"xe")
db = cx_Oracle.connect('pro2','pro2',dsn)
cursor = db.cursor()

In [3]:
# 이미지를 가져올 상품명을 조회하는 SQL문 생성
cursor.execute("select Product_name from product")

# SQL문으로 가져온 상품명을 리스트 A 로 담아줌
A = list(cursor.fetchall())

# 이미지가 NONE이면 기존 이미지주소를 사용할 SQL문 생성
cursor.execute("select IMG_ADDR from product")
B =  list(cursor.fetchall())

print(len(A), len(B))

400 400


In [4]:
## 크롤링해야되니까 크롬 드라이버 킴

url = 'https://www.kurly.com/shop/goods/goods_search.php?searched=Y&sword='
driver = wd.Chrome()
driver.get(url)

In [5]:
img_list = []
for i in range(60): # 한 페이지에 20개씩, 3개 페이지를 만들어야되니까 60번 돌도록
    search = driver.find_element_by_css_selector("#sword") #검색창 element 찾기
    time.sleep(1) # 찾았으면 쉬기
    search.send_keys(A[i]) # 검색창에 상품명 입력해주기
    search.send_keys(Keys.ENTER) # 검색
    time.sleep(3) # 이게 검색되고 시간이 좀 걸리더라고
    try :
        img = driver.find_element_by_css_selector("#goodsList > div.list_goods > div > ul > li:nth-child(1) > div > div > a > img")
        img_link = img.get_attribute("src")
        img_list.append(img_link) # 이미지 크롤링해서 img_list에 담아줬음
    except:
        img_link = "NONE" # 이미지가 없거나, 이미지 크롤링 되기 전 화면 넘어가면 걍 NONE
        img_list.append(img_link)

    time.sleep(1)
    search = driver.find_element_by_css_selector("#sword") # 검색창 다시 찾아줌
    search.send_keys(Keys.LEFT_SHIFT, Keys.HOME) # 검색했던 내용 다 지우는 코드 1
    search.send_keys(Keys.BACKSPACE) # 코드 2

In [10]:
# for문 돌려서 img_list 찾아진대로 DB에 연동해보자

for i in range(len(img_list)):
    a = str(A[i])
    b = str(img_list[i])
    if (b == 'NONE'): # 이미지 못 찾았으면 걍 원래 DB에 있던거 쓰도록
        b = str(B[i])
    sql = "update product set IMG_ADDR = :1 where PRODUCT_NAME= :2"
    val = (b,a)
    cursor.execute(sql,val)

InterfaceError: not open

In [11]:
# DB에 데이터 잘 옮겨졌는지 확인
cursor.execute("select * from product")
cursor.fetchall()

InterfaceError: not open

In [12]:
# 옮겨졌으면 commit 해야지
db.commit()

InterfaceError: not connected

In [13]:
# 그리고 데이터베이스 사용중지, 안 하면 DB사용자로 메모리 채우고 있음
cursor.close()
db.close()

InterfaceError: not open